In [57]:
import pandas as pd

In [58]:
#https://www.ine.es/jaxiT3/Datos.htm?t=2915
df_inhabitants=pd.read_csv('../data/inhabitants.csv',sep=';' , encoding='latin-1')
df_inhabitants.head()

,Comunidades y Ciudades Autónomas,Tamaño de los municipios,Periodo,Total
0,Total,Total,2021,47.385.107
1,Total,Total,2020,47.450.795
2,Total,Total,2019,47.026.208
3,Total,Total,2018,46.722.980
4,Total,Total,2017,46.572.132


In [59]:
#cogiendo datos a partir de 2015
df_inhabitants=df_inhabitants[df_inhabitants['Periodo']>=2015]         

In [60]:
#solo el mercado peninsular
df_inhabitants['Comunidades y Ciudades Autónomas'].unique()

array(['Total', '01 Andalucía', '02 Aragón', '03 Asturias, Principado de',
       '04 Balears, Illes', '05 Canarias', '06 Cantabria',
       '07 Castilla y León', '08 Castilla - La Mancha', '09 Cataluña',
       '10 Comunitat Valenciana', '11 Extremadura', '12 Galicia',
       '13 Madrid, Comunidad de', '14 Murcia, Región de',
       '15 Navarra, Comunidad Foral de', '16 País Vasco', '17 Rioja, La',
       '18 Ceuta', '19 Melilla'], dtype=object)

In [ ]:
to_exclude = ['Total','04 Balears, Illes','05 Canarias','18 Ceuta','19 Melilla']

In [66]:
df_inhabitants=df_inhabitants[~df_inhabitants['Comunidades y Ciudades Autónomas'].isin(to_exclude)]

In [68]:
#https://datos.madrid.es/portal/site/egob/
#menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=9f710c96da3f9510VgnVCM2000001f4a900aRCRD&vgnextchannel=
#374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default
df_calendario=pd.read_csv('../data/calendario.csv',sep=';' , encoding='latin-1')
df_calendario

,ï»¿DÃ­a,DÃ­a_semana,laborable / festivo / domingo festivo,Tipo de Festivo,Festividad
0,01/01/2013,martes,festivo,Festivo nacional,AÃ±o Nuevo
1,02/01/2013,miÃ©rcoles,laborable,NaN,NaN
2,03/01/2013,jueves,laborable,NaN,NaN
3,04/01/2013,viernes,laborable,NaN,NaN
4,05/01/2013,sÃ¡bado,sÃ¡bado,NaN,NaN
...,...,...,...,...,...
4279,NaN,NaN,NaN,NaN,NaN
4280,NaN,NaN,NaN,NaN,NaN
4281,NaN,NaN,NaN,NaN,NaN
4282,NaN,NaN,NaN,NaN,NaN
